In [1]:
%run download_and_preprocess_data.py

In [2]:
import pandas as pd

In [3]:
def list_compare(list1, list2):
    return set(list1) == set(list2)

In [4]:
cards = pd.read_json("data/cards-first-printing-scryfall.json")

# find the "middle-most" set(s) chronologically

In [5]:
set_list = cards.set_name.unique()
middle_index = len(set_list) // 2
if len(set_list) % 2 == 0:
    median_sets = [set_list[middle_index], set_list[middle_index + 1]]
else:
    median_sets = [set_list[middle_index]]
print(f"Median Set(s):{'/'.join(median_sets)}")

Median Set(s):Khans of Tarkir/Commander 2014


# find the most common card type

In [6]:
# Note: I have found if I go through these next few steps looking at all the cards, there may not be a card in the
# median sets that match all the criteria, so I am going to just look cards in the median sets and see where that
# gets me.

In [7]:
set_filter = (cards["set_name"].isin(median_sets))
type_count = cards[set_filter]["type1"].value_counts()
common_type = type_count.index[0]
print(f"Most Common Type: {common_type}")

Most Common Type: Creature 


# find the most common subtype

In [8]:
type_filter = set_filter & (cards["type1"] == common_type)
subtype_count = cards[type_filter]["subtype1"].value_counts()
common_subtype = subtype_count.index[0]
print(f"Most Common Subtype: {common_subtype}")

Most Common Subtype:  Human Warrior


# find the most common color identity

In [9]:
subtype_filter = type_filter & (cards["subtype1"] == common_subtype)
color_identity_count = cards[subtype_filter]["color_identity"].value_counts(dropna=False)
common_color_identity = color_identity_count.index[0]
print(f"Most Common Color Identity: {common_color_identity}")

Most Common Color Identity: ['W']


# find the most common rarity

In [10]:
color_identity_filter = (subtype_filter &
                         (cards["color_identity"].apply(list_compare, args=(common_color_identity,))))
rarity_count = cards[color_identity_filter]["rarity"].value_counts()
common_rarity = rarity_count.index[0]
print(f"Most Common Rarity: {common_rarity}")

Most Common Rarity: uncommon


# identify the best first guess card(s)

In [11]:
full_card_filter = color_identity_filter & (cards["rarity"] == common_rarity)
print(cards[full_card_filter][["name", "cmc", "color_identity", "rarity", "type1", "subtype1", "set_name"]])

                    name  cmc color_identity    rarity      type1  \
14240   Timely Hordemate  4.0            [W]  uncommon  Creature    
14255  Seeker of the Way  2.0            [W]  uncommon  Creature    

             subtype1         set_name  
14240   Human Warrior  Khans of Tarkir  
14255   Human Warrior  Khans of Tarkir  


# filter cards based on feedback from the game

In [12]:
current_filter = (
    (cards["cmc"] == 1.0) &
    (cards["color_identity"].apply(list_compare, args=(["W"],))) &
    (cards["rarity"] == "common") &
    (cards["type1"] == "Instant") &
    (cards["release_year"] > 2014) &
    (cards["release_year"] < 2016)
)
len(cards[current_filter])

2

In [13]:
card_list = cards[current_filter]["name"].unique()
middle_index = len(card_list) // 2
if len(set_list) % 2 == 0:
    median_cards = [card_list[middle_index - 1], card_list[middle_index]]
else:
    median_cards = [card_list[middle_index]]
cards[cards["name"].isin(median_cards)]

,name,cmc,color_identity,rarity,type1,subtype1,set_name,released_at,release_year
15089,Enshrouding Mist,1.0,[W],common,Instant,None,Magic Origins,2015-07-17,2015
15104,Lithomancer's Focus,1.0,[W],common,Instant,None,Battle for Zendikar,2015-10-02,2015
